In [58]:
import requests
import os
import pandas as pd
import numpy

In [11]:
def get_api_id():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_ID":
                return listing[1]

def get_api_secret():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_KEY":
                return listing[1]


In [81]:
def get_EVAbyName(station_names):
    url = "https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/stop-places/by-name/"
    headers = {
                "DB-Client-ID": get_api_id().replace("\'", ""),
                "DB-Api-Key": get_api_secret().replace("\'", ""),
                "accept": "application/vnd.de.db.ris+json"
            }
    eva_numbers = []

    for name in station_names:
        url = url + str(name)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            temp = []
            # get response
            temp.append(response.json())
            # now sort out that response is actually correct
            l = []
            for i in temp:
                for j in i['stopPlaces']:
                    l.append((j['evaNumber'], j['names']['DE']['nameLong']))
            # now remove all entries that are not correct:
            new_list=[li for li in l if li[1] == name]
            eva_numbers.append(new_list)
            
        else:
            print(f"Failed to fetch data for station {name}")
    
 
    # flatten
    return [x for xs in eva_numbers  for x in  xs]

In [113]:
def timetable(date, time, station_name, base_url="https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/"):
    eva_number = get_EVAbyName([station_name])[0][0]
    if(eva_number==[]):
        print(f"Failed to fetch station")
        return []
    headers = {
                "DB-Client-ID": get_api_id().replace("\'", ""),
                "DB-Api-Key": get_api_secret().replace("\'", ""),
                "accept": "application/xml"
            }
    # time in HH
    # date in YYMMDD
    url = base_url + str(eva_number) + "/" + str(date) + "/" + str(time)
    #url='https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/8000105/240523/08'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        timetable1=pd.read_xml(response.text, xpath=".//tl")
        timetable2=pd.read_xml(response.text, xpath=".//dp")
        timetable3=pd.read_xml(response.text, xpath=".//s")
        timetable = pd.concat([timetable1,timetable2,timetable3], axis=1)
    else:
        print(f"Failed to fetch timetable")
        timetable=[]
    return timetable

In [115]:
timetable("240523","09","Frankfurt(Main)Hbf")

C:\Users\ricat\AppData\Local\Temp\ipykernel_19244\3157251287.py:17: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  timetable1=pd.read_xml(response.text, xpath=".//tl")
C:\Users\ricat\AppData\Local\Temp\ipykernel_19244\3157251287.py:18: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  timetable2=pd.read_xml(response.text, xpath=".//dp")
C:\Users\ricat\AppData\Local\Temp\ipykernel_19244\3157251287.py:19: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  timetable3=pd.read_xml(response.text, xpath=".//s")


,f,t,o,c,n,pt,pp,ppth,l,wings,tra,id,tl,dp,ar
0,F,p,80,IC,2322,2.405231e+09,16.0,Frankfurt(Main)West|Bad Nauheim|Wetzlar|Dillen...,None,None,None,6512907201443103756-2405230926-1,NaN,NaN,NaN
1,D,p,R4,VIA,25124,2.405231e+09,18.0,Frankfurt-Niederrad|Frankfurt(M) Flughafen Reg...,2,None,None,-4158679146653019072-2405230820-9,NaN,NaN,NaN
2,N,p,800640,RE,4254,2.405231e+09,14.0,Frankfurt(Main)West|Bad Vilbel|Bad Vilbel-Gron...,34,None,None,2846214950676975755-2405230908-1,NaN,NaN,NaN
3,N,p,8005ND,RB,15710,2.405231e+09,21.0,Frankfurt-Höchst|Frankfurt-Unterliederbach|Lie...,RB12,None,None,-563620218745770846-2405230946-1,NaN,NaN,NaN
4,None,p,RSTN,STN,21467,2.405231e+09,9.0,Mannheim Hbf|Stuttgart Hbf|Ulm Hbf|Augsburg Hb...,None,None,None,-5301305442312220899-2405230917-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,N,p,800574,RE,4656,NaN,NaN,NaN,NaN,NaN,NaN,-2213558412361769980-2405230815-12,NaN,NaN,NaN
60,N,p,800772,RE,4609,NaN,NaN,NaN,NaN,NaN,NaN,-428509086416826477-2405230930-1,NaN,NaN,NaN
61,F,p,80,ICE,820,NaN,NaN,NaN,NaN,NaN,NaN,5509309536167095600-2405230553-5,NaN,NaN,NaN
62,F,p,80,ICE,1020,NaN,NaN,NaN,NaN,NaN,NaN,5781797999410176100-2405230511-8,NaN,NaN,NaN


In [90]:
type(get_EVAbyName("Frankfurt(Main)Hbf"))

list

In [4]:
api = ApiAuthentication(get_api_id().replace("\'", ""), get_api_secret().replace("\'", ""))
success: bool = api.test_credentials()

NameError: name 'ApiAuthentication' is not defined

In [ ]:
station_helper = StationHelper()
found_stations = station_helper.find_stations_by_lat_long(47.996713, 7.842174, 10)
found_stations_by_name = station_helper.find_stations_by_name("Freiburg")

In [ ]:
timetable_helper = TimetableHelper(YOUR_STATION_OBJECT, YOUR_API_AUTHENTICATION_OBJECT)
trains_in_this_hour = timetable_helper.get_timetable()
trains_at_given_hour = timetable_helper.get_timetable(12)

In [ ]:
trains_with_changes = timetable_helper.get_timetable_changes(trains)